# Le dataset :

Context
The Chernobyl disaster was a nuclear accident that occurred on 26 April 1986 at the No. 4 nuclear reactor in the Chernobyl Nuclear Power Plant, near the city of Pripyat in the north of the Ukrainian SSR. It is considered the worst nuclear disaster in history and is one of only two nuclear energy disasters rated at seven - the maximum severity - on the International Nuclear Event Scale, the other being the 2011 Fukushima Daiichi nuclear disaster in Japan.

Content
This dataset presents concentration of Iodine-131 (I-131), Caesium-134 (Cs-134) and Caesium-137 (radiocaesium, Cs-137) as anaerosol particles which were measured in specific location and date.
On each line, following information is given:

* country code
* locality name
* lattitude (degrees.hundredths of degrees)
* longitude (degrees.hundredths of degrees)
* date (year/month/day)
* hour of end of sampling (hours:minutes)
* duration (hours.minutes)
* I-131 concentration in Bq/m3 (aerosol particles)
* Cs-134 concentration in Bq/m3 (aerosol particles)
* Cs-137 concentration in Bq/m3 (aerosol particles)

Le but étant de prédire l'endroit où l'air est le plus pollué.

# Initialisation 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # graphs pyplot
import seaborn as sns # graphs
import geopandas as gpd # map

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.linear_model import LogisticRegression # regression logistique
from sklearn.model_selection import train_test_split

## Visualisation des données

In [ ]:
# lecture du dataset
df = pd.read_csv('../input/CHERNAIRedited.csv')

In [ ]:
# affichage bref du dataframe
df

In [ ]:
# exemples des premieres lignes
df.head()

In [ ]:
# comptage du nombre de valeurs pour chaque colonne
df.count()

On remarque qu'on a moins de données sur le Césium 134 & 137 que sur l'iode 131.

In [ ]:
# récupération des labels des colonnes
df.columns

In [ ]:
# description en termes de moyenne mini maxi des colonnes numériques
df.describe()

In [ ]:
# qq infos
df.info

## Tentatives de graphiques avec sns

In [ ]:
df['Cs 134 (Bq/m3)'] = pd.to_numeric(df['Cs 134 (Bq/m3)'],errors='coerce')
sns.jointplot("I 131 (Bq/m3)", "Cs 134 (Bq/m3)", df, kind='kde')

On regarde maintenant la distribution en Iode 131, Césium 134 et 137 :

In [ ]:
df['I 131 (Bq/m3)'] = pd.to_numeric(df['I 131 (Bq/m3)'],errors='coerce')
sns.distplot(df['I 131 (Bq/m3)'], color='blue')

In [ ]:
df['Cs 134 (Bq/m3)'] = pd.to_numeric(df['Cs 134 (Bq/m3)'],errors='coerce')
sns.distplot(df['Cs 134 (Bq/m3)'], color='blue')

In [ ]:
df['Cs 137 (Bq/m3)'] = pd.to_numeric(df['Cs 137 (Bq/m3)'],errors='coerce')
sns.distplot(df['Cs 137 (Bq/m3)'], color='blue')

On sait qu'on a plus de valeurs pour l'iode 131 mais les valeurs en Césium 137 sont plus concentrés.

## Tentons de créer une carte

On a une latitude et une longitude, nous allons essayer de faire une carte.

In [ ]:
print("La longitude mini est de : ")
print(df.X.min())

print("La longitude maxi est de : ")
print(df.X.max())

print("La latitude mini est de : ")
print(df.Y.min())

print("La latitude maxi est de :")
print(df.Y.max())

In [ ]:
df.plot(kind="scatter", x="X", y="Y", c="I 131 (Bq/m3)", cmap="rainbow", s=3, figsize=(12,12))


On affiche un graphique montrant l'évolution de la concentration en Iode 131 fonction de la fin de la date :

In [ ]:
df.groupby(['Date'])['I 131 (Bq/m3)'].mean().plot(kind = 'bar', figsize=(12,8))

On affiche un graphique montrant l'évolution de la concentration en Césium 134 fonction de la fin de la date :

In [ ]:
df.groupby(['Date'])['Cs 134 (Bq/m3)'].mean().plot(kind = 'bar', figsize=(12,8))

On affiche un graphique montrant l'évolution de la concentration en Césium 137 fonction de la fin de la date :

In [ ]:
df.groupby(['Date'])['Cs 137 (Bq/m3)'].mean().plot(kind = 'bar', figsize=(12,8))

On en conclue que l'iode 131 est plus concentrée que les autres et a eu un pic de concentration le 30 avril 1986, soit 4 jours après la catastrophe.

## Régression par forêts aléatoires

On va utiliser les forêts aléatoires pour la régression. On conserve l'ensemble des caractéristiques

In [ ]:
df1 = df.drop(['PAYS','Ville','Date','End of sampling','Duration(h.min)'], axis=1)
X = df1.drop(['I 131 (Bq/m3)'], axis=1)
y = df1['I 131 (Bq/m3)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

Il est difficile d'utiliser les méthodes de prédiction avec aussi peu de colonnes, nous devons enlever trop de paramètres...

## Faisons plutot une map avec Leaflet

On va faire une carte avec les différents endroits qui ont servi à des prélèvements :

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
#Define coordinates of where we want to center our map
boulder_coords = [47.583042, 12.70]

#Create the map
cher_map = folium.Map(location = boulder_coords, zoom_start = 5)

# Make an empty map
m = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)

# I can add marker one by one on the cher_map
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Y'], df.iloc[i]['X']], popup=df.iloc[i]['Ville']).add_to(cher_map)

#Display the map
cher_map